In [1]:
import csv
import re
import pandas as pd
import time
from random import randint
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Pre-define category links
main_categories = [
    {"Name": "Đồ Chơi - Mẹ & Bé", "URL": "https://tiki.vn/do-choi-me-be/c2549"},
    {"Name": "Điện Thoại - Máy Tính Bảng", "URL": "https://tiki.vn/dien-thoai-may-tinh-bang/c1789"},
    {"Name": "Làm Đẹp - Sức Khỏe", "URL": "https://tiki.vn/lam-dep-suc-khoe/c1520"},
    {"Name": "Điện Gia Dụng", "URL": "https://tiki.vn/dien-gia-dung/c1882"},
    {"Name": "Thời trang nữ", "URL": "https://tiki.vn/thoi-trang-nu/c931"},
    {"Name": "Thời trang nam", "URL": "https://tiki.vn/thoi-trang-nam/c915"},
    {"Name": "Giày - Dép nữ", "URL": "https://tiki.vn/giay-dep-nu/c1703"},
    {"Name": "Giày - Dép nam", "URL": "https://tiki.vn/giay-dep-nam/c1686"},
    {"Name": "Túi thời trang nữ", "URL": "https://tiki.vn/tui-vi-nu/c976"},
    {"Name": "Túi thời trang nam", "URL": "https://tiki.vn/tui-thoi-trang-nam/c27616"},
    {"Name": "Balo và Vali", "URL": "https://tiki.vn/balo-va-vali/c6000"},
    {"Name": "Phụ kiện thời trang", "URL": "https://tiki.vn/phu-kien-thoi-trang/c27498"},
    {"Name": "Đồng hồ và Trang sức", "URL": "https://tiki.vn/dong-ho-va-trang-suc/c8371"},
    {"Name": "Laptop - Máy Vi Tính - Linh kiện", "URL": "https://tiki.vn/laptop-may-vi-tinh-linh-kien/c1846"},
    {"Name": "Nhà Cửa - Đời Sống", "URL": "https://tiki.vn/nha-cua-doi-song/c1883"},
    {"Name": "Bách Hóa Online", "URL": "https://tiki.vn/bach-hoa-online/c4384"},
    {"Name": "Hàng Quốc Tế", "URL": "https://tiki.vn/hang-quoc-te/c17166"},
    {"Name": "Thiết Bị Số - Phụ Kiện Số", "URL": "https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815"},
    {"Name": "Voucher - Dịch vụ", "URL": "https://tiki.vn/voucher-dich-vu/c11312"},
    {"Name": "Ô Tô - Xe Máy - Xe Đạp", "URL": "https://tiki.vn/o-to-xe-may-xe-dap/c8594"},
    {"Name": "Nhà Sách Tiki", "URL": "https://tiki.vn/nha-sach-tiki/c8322"},
    {"Name": "Điện Tử - Điện Lạnh", "URL": "https://tiki.vn/dien-tu-dien-lanh/c4221"},
    {"Name": "Thể Thao - Dã Ngoại", "URL": "https://tiki.vn/the-thao-da-ngoai/c1975"},
    {"Name": "Máy Ảnh - Máy Quay Phim", "URL": "https://tiki.vn/may-anh/c1801"}
    ]

In [3]:
# Create a CSV file to store the scraped data
csv_file = open('tiki_scraped_data.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Category', 'Name', 'Price', 'Image', 'URL', 'Official', 'Stars', 'Sold Qty', 'Discount', 'Fast Shipping', 'Under Price', 'Installments', 'Gift'])

108

In [4]:
# Fetch HTML content from URL and save as parsed HTML in BeautifulSoup
def get_url(url):
    # Set the path to the ChromeDriver executable
    chromedriver_path = r"C:\Users\duong\OneDrive\Desktop\CoderSchool\Tiki_Web_Scraping\chromedriver.exe"

    # Configure Chrome options
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run Chrome in headless mode (without opening a browser window)

    # Start ChromeDriver service
    service = Service(chromedriver_path)

    # Start the WebDriver instance
    driver = webdriver.Chrome(service=service, options=options)

    # Navigate to the target URL
    driver.get(url)

    # Scroll to trigger lazy loading
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for all images to load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'img')))

    # Get the page source and create a BeautifulSoup object
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Close the WebDriver
    driver.quit()

    return soup

In [5]:
# Extract all products information from a certain category
def extract_all_products(url, category_name):
    soup = get_url(url)
    products = soup.find_all('a', {'class': 'product-item'})

    for product in products:
        try:
            # Name of product
            name_element = product.find('div', {'class': 'name'})
            name = name_element.find('h3').text if name_element else None

            # Price of product
            price = int(re.sub('\.', '', product.find('div', {'class': "price-discount__price"}).text[0:-1]))

            # Image URL of product
            img_element = product.find('div', {'class': 'image-wrapper'})
            picture_element = img_element.find('picture')
            srcset = picture_element.find('source')['srcset'] if picture_element and picture_element.find('source') else None
            if srcset:
                match = re.search(r'(https://\S+)', srcset)
                if match:
                    img = match.group(1)
            else:
                img = "No picture available"
            
            # Product URL
            purl = product['href']
            purl = product['href']
            if 'tiki.vn' in purl:
                purl = 'https:' + purl
            else:
                purl = 'https://tiki.vn' + purl

            # Check if this product has "Official" badge
            official = 'No'
            if product.find('div', {'class': 'thumbnail'}).img:
                official = 'Yes'

            # Reviewed stars
            stars_element = product.find('div', {'class': 'full-rating'})
            stars = float(stars_element.text) if stars_element else 'Not available'

            # Product sold
            sold_qty = 0
            if product.find('div', {'class': 'styles__StyledQtySold-sc-732h27-2 fCfYNm'}):
                sold_qty = int(product.find('div', {'class': 'styles__StyledQtySold-sc-732h27-2 fCfYNm'}).text[7:].replace('+', ''))

            # Check if product has any discount
            discount = 'None'
            if product.find('div', {'class': 'price-discount has-discount'}):
                discount = int(product.find('div', {'class': 'price-discount__discount'}).text.replace('-', '').replace('%', ''))

            # Check if product has "NOW" badge which means 2 hours shipping
            fast_shipping = 'No'
            if product.find('div', {'class': 'badge-delivery'}).img:
                fast_shipping = 'Yes'
            
            # Check if product is under-priced
            under_price = 'No'
            if product.find('div', {'class': 'badge-under-price'}):
                under_price = 'Yes'

            # Check if we can do payments
            installments = 'No'
            badge_under_rating = product.find('div', {'class': 'badge-under-rating'})
            if badge_under_rating and badge_under_rating.find('span') and badge_under_rating.find('span').text.strip() == 'Trả góp':
                installments = 'Yes'

            # Check for gifts
            gift = 'No'
            badge_under_rating = product.find('div', {'class': 'badge-under-rating'})
            if badge_under_rating and badge_under_rating.find('span') and badge_under_rating.find('span').text.strip() == 'Quà tặng':
                gift = 'Yes'

            # print(name, price, img, purl, official, stars, sold_qty, discount, fast_shipping, under_price, installments, gift)
            
            # Write the scraped data to the CSV file
            csv_writer.writerow([category_name, name, price, img, purl, official, stars, sold_qty, discount, fast_shipping, under_price, installments, gift])

        except Exception as err:
            print('ERROR WHILE SCRAPING', err)

    print(f'Number of products scraped from category "{category_name}": {len(products)}')


In [6]:
# Scrape data for each category and multiple pages
for category in main_categories:
    category_name = category['Name']
    category_url = category['URL']

    print(f"Scraping category: {category_name}")

    # Scrape the first 5  pages
    for page_number in range(1, 6):
        if page_number > 1:
            page = f'?page={page_number}'
            url = category_url + page
        else:
            url = category_url

        extract_all_products(url, category_name)
        time.sleep(randint(1, 2))

# Close the CSV file
csv_file.close()

Scraping category: Đồ Chơi - Mẹ & Bé
Number of products scraped from category "Đồ Chơi - Mẹ & Bé": 55
Number of products scraped from category "Đồ Chơi - Mẹ & Bé": 55
Number of products scraped from category "Đồ Chơi - Mẹ & Bé": 55
Number of products scraped from category "Đồ Chơi - Mẹ & Bé": 40
Number of products scraped from category "Đồ Chơi - Mẹ & Bé": 55
Scraping category: Điện Thoại - Máy Tính Bảng
Number of products scraped from category "Điện Thoại - Máy Tính Bảng": 55
Number of products scraped from category "Điện Thoại - Máy Tính Bảng": 55
Number of products scraped from category "Điện Thoại - Máy Tính Bảng": 55
Number of products scraped from category "Điện Thoại - Máy Tính Bảng": 40
Number of products scraped from category "Điện Thoại - Máy Tính Bảng": 40
Scraping category: Làm Đẹp - Sức Khỏe
Number of products scraped from category "Làm Đẹp - Sức Khỏe": 55
Number of products scraped from category "Làm Đẹp - Sức Khỏe": 55
Number of products scraped from category "Làm Đẹp - 